In [58]:
import pandas as pd
from sklearn.linear_model import LinearRegression
import statsmodels.formula.api as sm
import os
import numpy as np
from functools import reduce
import matplotlib as mpl
import sqlite3 as sql
%matplotlib inline

In [57]:
db_path = "/home/matt/Datasets/Battelle Data/database/main.db"

In [2]:
pd.set_option('max_colwidth',250)
pd.set_option('display.width',250)

In [3]:
os.chdir('/home/matt/Git/PGIP/src')
os.getcwd()

'/home/matt/Git/PGIP/src'

In [4]:
import pipeline as PGIP
os.chdir('..')
E = PGIP.Explorer('config/cache')
P = PGIP.Pipeline('config/cache')

In [5]:
E.readSampleData('sandbox')
E.renameColumns()

Data files: ['ISI_Medline-synthetic biology.csv', 'ISI_Medline-transcriptomics.csv']
column sample: ['PT' 'AN' 'DT' 'TI' 'FT' 'AU' 'CA' 'SO' 'VL' 'IS']
column sample: ['PT' 'AN' 'DT' 'TI' 'FT' 'AU' 'CA' 'SO' 'VL' 'IS']
Data files: ['ISI_Inspec-proteomics.csv', 'ISI_Inspec-transcriptomics.csv']
column sample: ['PT' 'AN' 'DT' 'PN' 'PC' 'TI' 'AU' 'ED' 'TR' 'OP']
column sample: ['PT' 'AN' 'DT' 'PN' 'PC' 'TI' 'AU' 'ED' 'TR' 'OP']
Data files: ['IEEE-proteomics.csv', 'IEEE-transcriptomics.csv']
column sample: ['Document Title' 'Authors' 'Author Affiliations' 'Publication Title'
 'Date Added To Xplore' 'Year' 'Volume' 'Issue' 'Start Page' 'End Page']
column sample: ['Document Title' 'Authors' 'Author Affiliations' 'Publication Title'
 'Date Added To Xplore' 'Year' 'Volume' 'Issue' 'Start Page' 'End Page']
Data files: ['EV-transcriptomics.xlsx', 'EV-proteomics.2013-2016.xlsx']
column sample: ['Title' 'Accession number' 'Author' 'Author affiliation'
 'Corresponding author' 'Source' 'Abbreviated 

In [6]:
df = pd.DataFrame()
for source in E.data:
    data = E.data[source].data
    data['Source'] = source
    df = pd.concat([df,data])

In [7]:
df.shape

(2385, 33)

In [8]:
df['Month'] = df.apply(P.adders.addMonth,axis=1)

In [9]:
df.Month.isnull().sum()

995

In [10]:
df.Year.isnull().sum()

0

In [11]:
df['TimeSlice'] = df.apply(P.adders.addTimeSlice,axis = 1)

In [12]:
df.TimeSlice.isnull().sum()

995

In [13]:
df.Issue = df.Issue.apply(P.cleaners.cleanIssue)

In [14]:
df.Volume = df.Volume.apply(P.cleaners.cleanVolume)

In [16]:
codens = set(df.CODEN.fillna('').unique())
def trim_coden(coden,codens):
    if pd.isnull(coden):
        return coden
    if len(coden) <= 5:
        return coden
    short = coden[0:5]
    if short in codens:
        return short
    else:
        return coden

In [17]:
df.CODEN = df.CODEN.apply(trim_coden,codens=codens)

In [18]:
def pred(row,params,target):
    
    if pd.isnull(row[target]):
        row.loc['Intercept'] = 1
        row = row[params.index]
        if any([pd.isnull(i) for i in row]):
            return np.nan
        
        p = row*params
        p = round(p.sum(),0)
        
        return int(p)
        
    else:
        return row.loc[target]

In [27]:
codens = df.CODEN.unique()
issns = df.ISSN.unique()
confidenceThreshold = .6

In [ ]:
primaryID = 'ISSN'
#secondaryIDs = ['CODEN','AbbrPubTitle','PublicationTitle']
secondaryIDs=['CODEN']

In [103]:
def impute_timeslice(df,primaryID,secondaryIDs,confidenceThreshold,verbose=False):
    total_imputed = 0
    imputed_indices = set()

    for code in df[primaryID].unique():
        a = df.loc[df[primaryID] == code,[primaryID]+secondaryIDs+['Issue','Volume','Year','Month','TimeSlice']]
        altcodes = {}
        for secondaryID in secondaryIDs:
            altcodes[secondaryID] = a[secondaryID].notnull().unique()

        locs = reduce(np.logical_or,[df[sec].isin(altcodes[sec]) for sec in secondaryIDs])
        #df.loc[locs,primaryID] = code
        if np.sum(locs) > 0:
            a = df.loc[locs,[primaryID]+secondaryIDs+['Issue','Volume','Year','Month','TimeSlice']]

        if a.shape[0] < 3:
            #rint('\n')
            continue

        count = a.TimeSlice.isnull().sum()

        if count == 0:
            #rint('\n')
            continue

        if (a.Issue.notnull() & a.Volume.notnull() & a.TimeSlice.notnull()).sum() >= 3:
            imputable = (a.Issue.notnull() & a.Volume.notnull() & a.TimeSlice.isnull()).sum()
            if imputable > 0:
                result = sm.ols(formula="TimeSlice ~ Volume + Issue", data=a).fit()
                target = 'TimeSlice'
        elif (a.Issue.notnull() & a.Month.notnull()).sum() >= 2:
            imputable = (a.Issue.notnull() & a.Month.isnull()).sum()
            if imputable > 0:
                result = sm.ols(formula="Month ~ Issue", data=a).fit()
                target = 'Month'
        elif (a.Volume.notnull() & a.TimeSlice.notnull()).sum() >= 2:
            imputable = (a.Volume.notnull() & a.TimeSlice.isnull()).sum()
            if imputable > 0:
                result = sm.ols(formula="TimeSlice ~ Volume", data=a).fit()
                target = 'TimeSlice'
        else:
            #print("imputable: 0")
            #rint('\n')
            continue

        if result.rsquared < 0 or result.rsquared > 1:
            #print(a)
            #rint('\n')
            continue

        if result.rsquared > confidenceThreshold:
            p = a.apply(pred,axis =1,**{'params':result.params,'target':target})
            new = set(a.index[a[target] != p])
            imputed_indices = imputed_indices.union(new)
            a.loc[:,target] = p
            #print(a)
            if target == 'Month':
                a.loc[:,'TimeSlice'] = a.apply(lambda row: 12*(row.loc['Year'] - 1990) + row.loc['Month'],axis=1)

            imputed = count - a.TimeSlice.isnull().sum()
            total_imputed += imputed
            
            if verbose:
                print("{}: {}".format(primaryID,code))
                print("sample size: {}".format(a.shape[0]))
                print("null count: {}".format(count))
                print("imputable: {}".format(imputable))
                print("R^2: {}\n".format(result.rsquared))
                print("IMPUTED: {}\n".format(imputed))
            
    return imputed_indices

# Let's try this on the whole db

In [99]:
df = pd.read_sql_query('select * from main',sql.connect(db_path))

In [109]:
df.shape

(115099, 35)

In [110]:
df.duplicated(subset=['Title']).sum()

34917

In [111]:
df.duplicated(subset=['DOI']).sum()

54242

In [124]:
imputed_indices = impute_timeslice(df,primaryID='ISSN',secondaryIDs=['CODEN','Title','DOI'],confidenceThreshold=.5)
total_imputed = len(imputed_indices)
print(total_imputed)

20432


# The verdict:

In [125]:
p_missing = df.TimeSlice.isnull().sum()/df.shape[0]
p_imputed = total_imputed/df.shape[0]
print("{}% missing".format(round(100*p_missing,3)))
print("{}% imputed".format(round(100*p_imputed,3)))
print("{}% missing after imputation".format(round(100*(p_missing-p_imputed),3)))

48.405% missing
17.752% imputed
30.654% missing after imputation


## And slightly worse final results if we subset on only the entries with unique DOI's

In [126]:
df2 = df[np.logical_not(df.duplicated(['DOI']))]
imputed_indices2=imputed_indices.intersection(set(df2.index))
total_imputed2 = len(imputed_indices2)
df2.shape

(60857, 35)

In [127]:
p_missing2 = df2.TimeSlice.isnull().sum()/df2.shape[0]
p_imputed2 = total_imputed2/df2.shape[0]
print("{}% missing".format(round(100*p_missing2,3)))
print("{}% imputed".format(round(100*p_imputed2,3)))
print("{}% missing after imputation".format(round(100*(p_missing2-p_imputed2),3)))

58.573% missing
23.623% imputed
34.951% missing after imputation


## functions for imputation in the db:

In [109]:
def imputeCODEN(connection,table,recordID=None):
    if not recordID:
        recordID = 'ROWID'
    columns = [recordID,'CODEN','AbbrPubTitle','PublicationTitle']
    columnString = ', '.join(columns)

    codens = connection.execute('SELECT DISTINCT CODEN FROM {0}'.format(table)).fetchall()
    codens = list(next(zip(*codens)))
    
    for code in codens:
        
        command = 'SELECT {0} FROM {1} WHERE CODEN = "{3}"'.format(columnString,table,code)
        df = connection.execute(command).fetchall()
        df = pd.DataFrame(df,columns = columns)
        
        abbrevs = list(df.AbbrPubTitle.unique())
        titles = list(df.PublicationTitle.unique())

        if len(abbrevs) > 1 or len(titles) > 1:
            # Update the db
            abbrString = '"' + '","'.join(abbrevs) + '"'
            titleString = '"' + '", "'.join(titles) + '"'
            
            command = ('UPDATE {0} SET CODEN = "{1}" ' +
                      'WHERE AbbrPubTitle IN ({2}) ' +
                      'OR PublicationTitle IN ({3})').format(table,code,abbrString,titleString)
            
            mem.execute(command)
        

def imputeTimeSlice(connection,table,recordID=None,confidenceThreshold = .6):
    if not recordID:
        recordID = 'ROWID'
    columns = [recordID,'Issue','Volume','Year','Month','TimeSlice']
    columnString = ', '.join(columns)
        
    codens = connection.execute('SELECT DISTINCT CODEN FROM {0}'.format(table)).fetchall()
    codens = list(next(zip(*codens)))
    
    for code in codens:
        command = 'SELECT {0} FROM {1} WHERE CODEN = "{3}"'.format(columnString,table,code)
        df = connection.execute(command).fetchall()
        df = pd.DataFrame(df,columns = columns) 

        if df.Timeslice.isnull().sum() == 0 or df.shape[0] < 2:
            continue     
        
        df = imputeTimeSliceSample(df)
        df = df[df.imputed,[recordID,'Month','TimeSlice']]
        df = df.setindex(recordID)
        
        for i in df.index:
            command = ('UPDATE {0} SET Month = {1}, TimeSlice = {2} ' + 
                      'WHERE {3} = {4}').format(table,df.loc[i,'Month'],df.loc[i,'TimeSlice'],recordID = i)
            connection.execute(command)
            
        


def imputeTimeSliceSample(dataframe):
        a = dataframe
        a.loc[:,'imputed'] = False
        initial = a.loc[:,'TimeSlice']
        
        if a.shape[0] < 2:
            return a
            
        if (a.Issue.notnull() & a.Volume.notnull() & a.TimeSlice.notnull()).sum() >= 3:
            imputable = (a.Issue.notnull() & a.Volume.notnull() & a.TimeSlice.isnull()).sum()
            if imputable > 0:
                result = sm.ols(formula="TimeSlice ~ Volume + Issue", data=a).fit()
                target = 'TimeSlice'
        elif (a.Issue.notnull() & a.Month.notnull()).sum() >= 2:
            imputable = (a.Issue.notnull() & a.Month.isnull()).sum()
            if imputable > 0:
                result = sm.ols(formula="Month ~ Issue", data=a).fit()
                target = 'Month'
        elif (a.Volume.notnull() & a.TimeSlice.notnull()).sum() >= 2:
            imputable = (a.Volume.notnull() & a.TimeSlice.isnull()).sum()
            if imputable > 0:
                result = sm.ols(formula="TimeSlice ~ Volume", data=a).fit()
                target = 'TimeSlice'
        else:
            return a
        
        if result.rsquared > confidenceThreshold:
            p = a.apply(pred,axis =1,**{'params':result.params,'target':target})
            a.loc[:,target] = p
            print(a)
            if target == 'Month':
                a.loc[:,'TimeSlice'] = a.apply(lambda row: 12*(row.loc['Year'] - 1990) + row.loc['Month'])
            
            a.loc[:,'imputed'] = a.TimeSlice != initial
        
        return imputed
            